In [1]:
from secret_key import openai_api_key
import os
os.environ["OPENAI_API_KEY"]=openai_api_key

In [6]:
from langchain.llms import OpenAI

In [7]:
llm=OpenAI(temperature=0.5)

### Prompt Template and Chain

In [9]:
from langchain.prompts import PromptTemplate

In [11]:
from langchain.chains import LLMChain

In [40]:
name_template=PromptTemplate(
    input_variables=['cuisine', 'type', 'place'],
    template='''
    I am planning to open a restaurant. I'll provide you with the details below, Suggest a fancy and catchy name for it. (Give the name with a relevant emoji)

    Type: {type}
    Cuisine: {cuisine}
    Location: {place}
    '''
)

In [41]:
name_chain=LLMChain(llm=llm,prompt=name_template,output_key="restaurant_name")

In [64]:
menu_template=PromptTemplate(
    input_variables=['restaurant_name','von','menu'],
    template="Give me a comma separated values of menu items (minimum 20, maximum 50) for the {von} restaurant named {restaurant_name} which serves {menu}"
)

In [46]:
menu_chain=LLMChain(llm=llm,prompt=menu_template,output_key="menu_items")

In [48]:
price_template=PromptTemplate(
    input_variables=['menu_items','location'],
    template='''
    Give me a comma separated price list for the below mentioned menu items.
    
    {menu_items}
    
    The restaurant is located in {location}. Assign the price accordingly. Format the output as a comma separated values.
    '''
)

In [49]:
price_chain=LLMChain(llm=llm,prompt=price_template,output_key="price")

In [50]:
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

In [62]:
def generate_name_and_menu(res_type,cuisine,place):
    overall_chain = SequentialChain(
        memory=SimpleMemory(memories={"menu": cuisine, "location":place, "von":res_type}),
        chains=[name_chain, menu_chain, price_chain],
        input_variables=["cuisine", "type","place"],
        output_variables=["restaurant_name","menu_items","price"]
    )
    
    output=overall_chain({"cuisine":cuisine,"type":res_type,"place":place})
    print(output)

In [63]:
generate_name_and_menu("Vegetarian","South India","Coimbatore")

{'cuisine': 'South India', 'type': 'Vegetarian', 'place': 'Coimbatore', 'menu': 'South India', 'location': 'Coimbatore', 'von': 'Vegetarian', 'restaurant_name': "\n🍛  Veggie's Delight, Coimbatore 🍛", 'menu_items': ' Cuisine\n\nIdli, Dosa, Vada, Upma, Pongal, Uttapam, Sambar Rice, Curd Rice, Masala Dosa, Rava Idli, Onion Rava Dosa, Onion Uttapam, Tomato Uttapam, Onion Pakoda, Veg Biryani, Veg Fried Rice, Veg Pulao, Veg Manchurian, Veg Kurma, Veg Korma, Aloo Gobi, Aloo Mutter, Aloo Jeera, Veg Kolhapuri, Veg Chettinadu, Veg Biryani, Veg Jalfrezi, Veg Kofta Curry, Veg Cutlet, Veg Fried Rice, Veg Sweet Corn Soup, Veg Manchow Soup, Veg Clear Soup, Veg Noodles, Veg Fried Noodles, Veg Chowmein, Veg Hakka Noodles.', 'price': '\nIdli - 25, Dosa - 40, Vada - 20, Upma - 30, Pongal - 35, Uttapam - 35, Sambar Rice - 30, Curd Rice - 35, Masala Dosa - 50, Rava Idli - 40, Onion Rava Dosa - 50, Onion Uttapam - 40, Tomato Uttapam - 40, Onion Pakoda - 25, Veg Biryani - 90, Veg Fried Rice - 70, Veg Pulao -